In [ ]:
import re
import ast
import time
# import torch
from tqdm import tqdm
import tensorflow_hub as hub
# import tensorflowtext
# import matplotlib.pyplot as plt
# from nltk.tokenize import word_tokenize
tqdm.pandas()
# from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# from transformers import BertTokenizer, BertModel

In [ ]:
toxic_classification_test = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv")
toxic_classification_test_labels = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv")
toxic_classification_train = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv")

data_path = "/kaggle/input/jigsaw-toxic-severity-rating/"
validation_data = pd.read_csv(data_path+"validation_data.csv")
data = pd.read_csv(data_path+"comments_to_score.csv")

In [ ]:
embed = hub.load("/kaggle/input/universalsentenceencoder4/")

In [ ]:
def clean(text):
    text = text.replace("\n", "")
    text = " ".join(text.split())
    text = re.sub(r'[^\w\s]','', text)
    text = re.sub(r'[0-9]','', text)
    return text

In [ ]:
df_test = pd.merge(toxic_classification_test, toxic_classification_test_labels, how='left', on='id')
df = pd.concat([toxic_classification_train, df_test])

df = df[(df['toxic']!=-1) & (df['severe_toxic']!=-1) & (df['obscene']!=-1) & (df['threat']!=-1) & (df['insult']!=-1) & (df['identity_hate']!=-1)]
df['toxicity'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1)
df['toxicity'] = df['toxicity'].apply(lambda x: 4 if x in [4,5,6] else x)

In [ ]:
sampled_df = df.groupby('toxicity', group_keys=False).apply(lambda x: x.sample(min(len(x), 4000)))
sampled_df['cleaned_text'] = sampled_df['comment_text'].progress_apply(clean)
del(sampled_df['comment_text'])

In [ ]:
start = time.time()
use_vecs = embed(sampled_df.cleaned_text.values)['outputs'].numpy()
end = time.time()

print("Time taken to embed:", end-start)

In [ ]:
# import xgboost as xgb
import lightgbm as lgb

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(use_vecs, sampled_df.toxicity.values, test_size=1/4, random_state=0)

In [ ]:
model = lgb.LGBMClassifier(learning_rate=0.05,random_state=42)
model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_train,y_train)],
          verbose=20,eval_metric='logloss')

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
def get_rank_score(vecs):
#     vec = vectorizer.transform(corpus).toarray()
    labels = model.predict(vecs)
    prob_scores = model.predict_proba(vecs)
    rank_scores = labels + np.max(prob_scores, axis=1)
    return rank_scores

In [ ]:
data['text'] = data.text.apply(clean)
data['bert_vec'] = data.text.apply(get_bert_embedding)
data['score'] = get_rank_score(data.bert_vec.values)
del(data['text'])
del(data['bert_vec'])

## Final tuned model

In [ ]:
lgb_model = lgb.LGBMClassifier(learning_rate=0.1,num_leaves=70, 
                   min_data_in_leaf=600, max_depth=10, 
                   random_state=42, num_iterations=200)
lgb_model.fit(use_vecs,sampled_df.toxicity.values)